Connecting to Thymio

In [2]:
import tdmclient.notebook
await tdmclient.notebook.start()

In order to use sync_to_async

In [3]:
from asgiref.sync import sync_to_async

In order to do the tests:

In [4]:
test_functions = True

# Functions in order to test the local avoidance

In [5]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

if test_functions:
    motors(100, 100) #test with lower speed value
    sleep(2)
    motors(0, 0)

# Local Navigation

### Global variables and constants

In [6]:
LED = 32
SMALL_LED = 2

left_obstacle = False
right_obstacle = False
Bloqued = False
side = 2 #0 : Thymio turned left / 1: Thymio turned right


In [7]:
@tdmclient.notebook.sync_var
def light_em_up(left=0,right=0,wait=0):
    global leds_top, leds_buttons, leds_circle
    leds_top = [0,LED,LED]
    if(wait):
        leds_circle = [SMALL_LED, SMALL_LED, LED, SMALL_LED, SMALL_LED, SMALL_LED, LED, SMALL_LED]
    else:
        leds_circle = [0, 0, 0, 0, 0, 0, 0, 0]
        

### Check cars
This function will check if there's something in front of Thymio. If there's, it will return **TRUE** and take the control of the Thymio. If there's nothing, it will return **FALSE** and let the control to optimal path

In [8]:
@tdmclient.notebook.sync_var
def check_cars(state=1, Tres_high=800, Tres_mid_side_high=1500, Tres_low=1500, Tres_mid_side_low=1000):
    global prox_horizontal

 
    if((prox_horizontal[2]>Tres_high)or(prox_horizontal[1]>Tres_mid_side_high)or(prox_horizontal[3]>Tres_mid_side_high)):
        return True        
    #There's something in front of Thymio --> avoid_function take the control
    else:
        return False
    #There's nothting, Thymio continue its normal ways 

In [9]:
if(test_functions):
    t = True
    i = 0
    while(t):
        print(check_cars())
        sleep(1)
        i = i+1
        if i>10:
            t = False

True
True
True
True
True
False
False
False
False
False
False


### Avoid function
This function will check to the left/right if there's a "Thymio-car" so our Thymio can avoid the car in front of it. If there's a "Thymio-car" in front of it and on its left/right, our Thymio will wait until the way to the left/right is free again.

**We call this function when check_cars() = True otherwise we call the logical path**

In [25]:
async def avoid_obstacle_2(Tres_side_high=1000, Tres_side_low=800, counter=9999):
    global left_obstacle, right_obstacle, prox_horizontal, Bloqued, leds_circle, side
    speed0 = 100       # nominal speed
    obstSpeedGain = 20  # /100 (actual gain: 5/100=0.05)
    
    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[0], prox_horizontal[4]]
    
    #check left
    if(obst[0]>Tres_side_high):
        left_obstacle = True
    elif(obst[0]<Tres_side_low):
        left_obstacle = False
        
    #check right
    if(obst[1]>Tres_side_high):
        right_obstacle = True
    elif(obst[1]<Tres_side_low):
        right_obstacle = False
    
    #if both sides are bloqued --> wait
    if((right_obstacle)and(left_obstacle)):
        await sync_to_async(light_em_up)(0,0,1)
        Bloqued = True
    #otherwise avoid the object:    
    else:
        await sync_to_async(light_em_up)(0,0,0)
        Bloqued = False
        
    speed_l = speed0 + obstSpeedGain * (obst[0] // 100)
    speed_r = speed0 + obstSpeedGain * (obst[1] // 100)
    
    if(left_obstacle):
        speed_r = 0
    if(right_obstacle):
        speed_l = 0
    if not (left_obstacle*right_obstacle):
        if (speed_l > speed_r):
            speed_r = 0
        else:
            speed_l = 0
        
    motors(speed_l,speed_r)
    
    #If Thymio is turning right
    if(side==1):
        #If there's no more obstacle to the left --> go back to your path!
        if(obst[0]<Tres_side_low):
            side = 2
            return True
    #If Thymio is turning right    
    elif(side==0):
        #If there's no more obstacle to the left --> go back to your path!
        if(obst[2]<Tres_side_low):
            side = 2
            return True        
    
    #to do the counter later!

In [24]:
async def avoid_obstacle(Tres_side_high=1000, Tres_side_low=800, counter=9999):
    global left_obstacle, right_obstacle, prox_horizontal, Bloqued, leds_circle, side
    speed0 = 100       # nominal speed
    obstSpeedGain = 20  # /100 (actual gain: 5/100=0.05)
    
    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[0], prox_horizontal[4]]
    
    #check left (in order to know if Thymio is bloqued)
    if(obst[0]>Tres_side_high):
        left_obstacle = True
    elif(obst[0]<Tres_side_low):
        left_obstacle = False
        
    #check right (in order to know if Thymio is bloqued)
    if(obst[1]>Tres_side_high):
        right_obstacle = True
    elif(obst[1]<Tres_side_low):
        right_obstacle = False
    
    #if both sides are bloqued --> wait
    if((right_obstacle)and(left_obstacle)):
        await sync_to_async(light_em_up)(0,0,1)
        Bloqued = True
    #otherwise avoid the object:    
    else:
        await sync_to_async(light_em_up)(0,0,0)
        Bloqued = False
        
    speed_l = speed0 + obstSpeedGain * (obst[0] // 100)
    speed_r = speed0 + obstSpeedGain * (obst[1] // 100)
    

    if (speed_l > speed_r):
        speed_r = 0
    else:
        speed_l = 0
        
    motors(speed_l,speed_r)
    
    #If Thymio is turning right
    if(side==1):
        #If there's no more obstacle to the left --> go back to your path!
        if(obst[0]<Tres_side_low):
            side = 2
            return True
    #If Thymio is turning right    
    elif(side==0):
        #If there's no more obstacle to the left --> go back to your path!
        if(obst[2]<Tres_side_low):
            side = 2
            return True        
    
    #to do the counter later!

In [21]:
async def control():
    while True:
        if(check_cars()):
            await avoid_obstacle()
        else:
            motors(100,100)
        sleep(0.2)

In [ ]:
if(test_functions):
    light_em_up()
    
    await control()
    

In [ ]:
motors(0,0)